In [ ]:
Content is user-generated and unverified.


Loading is taking longer than expected
There may be an issue with the content you’re trying to load.
The code itself may still be valid and functional.

#!/usr/bin/env python3
"""
Modularity Tester - Test whether a q-expansion might be modular

Usage:
    python modularity_tester.py

Then modify the coeffs, k, and N variables below with your data.
"""

import numpy as np
from fractions import Fraction
from math import gcd, floor, sqrt

# ============================================================================
# USER INPUT SECTION - MODIFY THESE
# ============================================================================

# Your q-expansion coefficients [a_0, a_1, a_2, a_3, ...]
# The more coefficients you provide, the better the tests
coeffs = [
    1, -5, -70/3, 28219/12, 1702183/30, 84674353/90, 
    223590851/63, -251901250975/1008, -203241992963123/22680, 
    -2549624882458823/16200
]

# Hypothesized weight and level
# If you don't know, the script can try multiple hypotheses (see bottom)
k = 12  # weight
N = 1   # level

# How many primes to test Hecke operators with (default: 10)
max_primes_to_test = 10

# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def is_prime(n):
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    for i in range(3, int(sqrt(n)) + 1, 2):
        if n % i == 0:
            return False
    return True

def primes_up_to(n):
    return [p for p in range(2, n+1) if is_prime(p)]

def dimension_cusp_forms(k, N):
    """Rough estimate of dim S_k(Γ_0(N))"""
    if k < 2 or k % 2 != 0:
        return 0
    if N == 1:
        return max(0, k // 12 - 1) if k % 12 == 2 else k // 12
    g = 1 + N/12
    return max(0, floor((k-1) * g))

def apply_hecke_T_p(coeffs, p, k):
    """Apply Hecke operator T_p to q-expansion"""
    max_n = len(coeffs)
    result = [0] * max_n
    
    for n in range(max_n):
        if n * p < max_n:
            result[n] += coeffs[n * p]
        if n > 0 and n % p == 0:
            result[n] += (p ** (k-1)) * coeffs[n // p]
    
    return result

def extract_eigenvalue(original_coeffs, hecke_coeffs, n):
    for i in range(1, min(n, len(original_coeffs))):
        if abs(original_coeffs[i]) > 1e-10:
            return hecke_coeffs[i] / original_coeffs[i]
    return None

def check_multiplicativity(eigenvalues, primes, tolerance=1e-6):
    checks_passed = 0
    checks_failed = 0
    failures = []
    
    for i, p in enumerate(primes):
        for q in primes[i+1:]:
            if p * q not in eigenvalues:
                continue
            expected = eigenvalues[p] * eigenvalues[q]
            actual = eigenvalues[p * q]
            
            if abs(actual - expected) < tolerance:
                checks_passed += 1
            else:
                checks_failed += 1
                failures.append(f"λ_{p}*λ_{q} ≠ λ_{p*q}: {eigenvalues[p]:.6f} * {eigenvalues[q]:.6f} = {expected:.6f} but λ_{p*q} = {actual:.6f}")
    
    return checks_passed, checks_failed, failures

def ramanujan_bound_check(coeffs, k, max_check=100):
    """Check |a_p| ≤ 2p^{(k-1)/2} for prime p"""
    violations = []
    
    for p in primes_up_to(min(len(coeffs)-1, max_check)):
        if p >= len(coeffs):
            break
        bound = 2 * (p ** ((k-1)/2))
        if abs(coeffs[p]) > bound * 1.1:
            violations.append(f"|a_{p}| = {abs(coeffs[p]):.4f} > {bound:.4f}")
    
    return violations

# ============================================================================
# MAIN TEST FUNCTION
# ============================================================================

def test_modularity(coeffs, k, N, max_primes=10):
    """
    Test if q-expansion appears modular
    
    Returns: (score, max_score, detailed_results)
    """
    print(f"\n{'='*70}")
    print(f"MODULARITY TEST: Weight {k}, Level {N}")
    print(f"{'='*70}")
    print(f"Number of coefficients: {len(coeffs)}")
    print(f"First 10 coefficients:")
    for i in range(min(10, len(coeffs))):
        if isinstance(coeffs[i], (int, float)):
            try:
                frac = Fraction(coeffs[i]).limit_denominator(10000)
                print(f"  a_{i} = {frac} = {float(frac):.6f}")
            except:
                print(f"  a_{i} = {coeffs[i]:.6f}")
    print()
    
    results = {}
    score = 0
    
    # Test 1: Ramanujan bound
    print(f"[1] Ramanujan-Petersson Bound (|a_p| ≤ 2p^{{(k-1)/2}})")
    print(f"-" * 70)
    violations = ramanujan_bound_check(coeffs, k)
    if violations:
        print(f"❌ {len(violations)} violations found")
        for v in violations[:5]:
            print(f"   {v}")
    else:
        print(f"✓ PASSED: All coefficients satisfy bound")
        score += 1
    results['ramanujan'] = len(violations) == 0
    print()
    
    # Test 2: Dimension
    print(f"[2] Dimension Check")
    print(f"-" * 70)
    dim = dimension_cusp_forms(k, N)
    print(f"Estimated dim S_{k}(Γ_0({N})) ≈ {dim}")
    if dim > 0:
        print(f"✓ Space is non-trivial (good)")
    else:
        print(f"⚠ Space appears trivial - unusual for this weight/level")
    print()
    
    # Test 3: Hecke operators
    print(f"[3] Hecke Eigenvalue Relations")
    print(f"-" * 70)
    
    primes = primes_up_to(50)[:max_primes]
    eigenvalues = {}
    
    for p in primes:
        hecke_result = apply_hecke_T_p(coeffs, p, k)
        lambda_p = extract_eigenvalue(coeffs, hecke_result, 20)
        if lambda_p is not None:
            eigenvalues[p] = lambda_p
    
    # Check if λ_p = a_p (eigenform property)
    matches = all(
        abs(eigenvalues[p] - coeffs[p]) < 1e-6 
        for p in eigenvalues if p < len(coeffs)
    )
    
    if matches:
        print(f"✓ All λ_p match a_p (eigenform property)")
        score += 1
    else:
        print(f"❌ Eigenvalues don't match coefficients")
        print(f"Sample eigenvalues:")
        for p in list(eigenvalues.keys())[:5]:
            a_p = coeffs[p] if p < len(coeffs) else 0
            print(f"  λ_{p} = {eigenvalues[p]:.6f}, a_{p} = {a_p:.6f}")
    
    results['hecke_matches'] = matches
    
    # Check multiplicativity
    passed, failed, failures = check_multiplicativity(eigenvalues, list(eigenvalues.keys()))
    print(f"\nMultiplicativity (λ_p·λ_q = λ_{{pq}}):")
    print(f"  Passed: {passed}, Failed: {failed}")
    
    if failed == 0 and passed > 0:
        print(f"✓ Multiplicativity satisfied")
        score += 1
    elif failed > 0:
        print(f"❌ Multiplicativity violations:")
        for f in failures[:3]:
            print(f"  {f}")
    
    results['mult_passed'] = passed
    results['mult_failed'] = failed
    print()
    
    # Test 4: Rationality
    print(f"[4] Coefficient Rationality")
    print(f"-" * 70)
    try:
        rational_coeffs = [Fraction(c).limit_denominator(10**10) for c in coeffs[:20]]
        all_rational = all(abs(float(frac) - coeffs[i]) < 1e-10 for i, frac in enumerate(rational_coeffs))
        
        if all_rational:
            print(f"✓ Coefficients appear rational")
            score += 1
            denoms = [frac.denominator for frac in rational_coeffs[1:11]]
            print(f"  Denominators: {denoms}")
        else:
            print(f"❌ Coefficients not rational")
    except:
        print(f"⚠ Could not check rationality")
    print()
    
    # Summary
    max_score = 4
    print(f"{'='*70}")
    print(f"OVERALL SCORE: {score}/{max_score}")
    print(f"{'='*70}")
    
    if score == max_score:
        print("✓✓✓ STRONG EVIDENCE - Likely modular")
    elif score >= 3:
        print("✓✓ MODERATE EVIDENCE - May be modular")
    elif score >= 2:
        print("⚠ WEAK EVIDENCE - Probably not modular")
    else:
        print("✗ UNLIKELY TO BE MODULAR")
    print(f"{'='*70}\n")
    
    return score, max_score, results

# ============================================================================
# RUN THE TEST
# ============================================================================

if __name__ == "__main__":
    print("\n" + "="*70)
    print("MODULARITY TESTER")
    print("="*70)
    
    # Single hypothesis test
    score, max_score, results = test_modularity(coeffs, k, N, max_primes_to_test)
    
    # Uncomment below to test multiple (k, N) hypotheses
    """
    print("\n" + "="*70)
    print("TESTING MULTIPLE HYPOTHESES")
    print("="*70)
    
    hypotheses = [
        (2, 1),
        (4, 1),
        (6, 1),
        (12, 1),
        (2, 11),
        (2, 13),
    ]
    
    best_score = -1
    best_hyp = None
    
    for k_test, N_test in hypotheses:
        score, max_score, _ = test_modularity(coeffs, k_test, N_test, max_primes=5)
        if score > best_score:
            best_score = score
            best_hyp = (k_test, N_test)
    
    print(f"\nBEST HYPOTHESIS: k={best_hyp[0]}, N={best_hyp[1]} (score {best_score}/{max_score})")
    """
